In [23]:
import pandas as pd
import numpy as np

# Filter all warnings. If you would like to see the warnings, please comment the two lines below.
import warnings
warnings.filterwarnings('ignore')


import pandas as pd
import numpy as np
import re

gdp_years = [str(x) for x in range(2006, 2016)]

def clean_names(x):
    nums = re.match('(\D*)\d*', x)
    if nums:
        x = nums.group(1)

    if x == "Republic of Korea": return "South Korea"
    if x == "United States of America": return "United States"
    if x == "United Kingdom of Great Britain and Northern Ireland": return "United Kingdom"
    if x == "China, Hong Kong Special Administrative Region": return "Hong Kong"
    
    pars = re.match('(.*)\(.*\)', x)
    if pars: return pars.group(1).strip()
    
    return x


In [118]:
#GDP = pd.read_csv('assets/gdp.csv')
GDP = pd.read_csv('assets/gdp.csv',  skiprows=4)
#GDP.head()
#GDP.tail(4)
#GDP[30:35]
#GDP.loc[100:105, '2000':'2005']
#GDP[['Country Name', 'Country Code']]
#GDP['Population'] = GDP['2015'].mean()
#GDP
#GDP['1960'] == np.nan
#import math
#math.isnan(np.nan)
#~GDP['1960'].apply(math.isnan)
#GDP[~GDP['1960'].isnull()]

#GDP.T
# Multilevel index, stack, unstack, 

GDP['Country Name'].replace(["Korea, Rep.", "Iran, Islamic Rep.", "Hong Kong SAR, China"], 
                            ["South Korea", "Iran", "Hong Kong"],
                            inplace=True)
GDP.rename({'Country Name': 'Country'}, inplace=True, axis='columns')
GDP = GDP.filter(items=['Country'] + gdp_years)


In [119]:


energy = pd.read_excel('assets/energy_indicators.xls',
                           skiprows=17, 
                           skipfooter=38,
                           na_values=['...']).iloc[:, 2:]
energy.rename({'Unnamed: 2': 'Country',
               'Petajoules': 'Energy Supply',
               'Gigajoules': 'Energy Supply per Capita',
               '%': '% Renewable'}, inplace=True, axis='columns')
energy['Energy Supply'] = energy['Energy Supply'].apply(lambda x: x * 1_000_000)
energy['Country'] = energy['Country'].apply(clean_names)


science = pd.read_excel('assets/sci_journal_rank.xlsx')


In [122]:


def merged():  
    return science[:15].merge(energy, how='inner', on='Country').merge(GDP, how='inner', on='Country').set_index('Country')


merged()

#merged().groupby('Region').first()
#merged().groupby('Region').mean()
#merged().groupby('Region').describe()


,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Region,,,,,,,,,,,,,,,,,,,,
Asiatic Region,4.5,86125.5,85364.0,748611.5,398240.5,9.6925,180.5,4.759425e+10,122.25,11.809041,2.318996e+12,2.613739e+12,2.969613e+12,3.154729e+12,3.651736e+12,4.196308e+12,4.460377e+12,4.488410e+12,4.712386e+12,4.755097e+12
Eastern Europe,7.0,31880.0,31664.0,91906.0,42529.0,2.8800,84.0,3.070900e+10,214.00,17.288680,9.899305e+11,1.299706e+12,1.660846e+12,1.222644e+12,1.524917e+12,2.045926e+12,2.208296e+12,2.292473e+12,2.059242e+12,1.363481e+12
Latin America,15.0,18024.0,17745.0,152380.0,37112.0,8.4500,121.0,1.214900e+10,59.00,69.648030,1.107640e+12,1.397084e+12,1.695825e+12,1.667020e+12,2.208872e+12,2.616201e+12,2.465189e+12,2.472807e+12,2.455994e+12,1.802214e+12
Middle East,12.0,19371.0,19085.0,242250.0,77012.0,12.5100,127.0,9.172000e+09,119.00,5.707721,2.662989e+11,3.498816e+11,4.123362e+11,4.163970e+11,4.868076e+11,5.807649e+11,5.988685e+11,4.602931e+11,4.326870e+11,3.849515e+11
Northern America,5.0,93722.0,91649.5,1216015.0,364166.0,14.4450,271.5,5.063450e+10,291.00,36.758205,7.565013e+12,7.958418e+12,8.130988e+12,7.910043e+12,8.302759e+12,8.665615e+12,9.012848e+12,9.316029e+12,9.665348e+12,9.890417e+12
Pacific Region,14.0,18077.0,17661.0,263733.0,44799.0,14.5900,161.0,5.386000e+09,231.00,11.810810,7.460542e+11,8.530996e+11,1.053996e+12,9.278052e+11,1.146138e+12,1.396650e+12,1.546152e+12,1.576184e+12,1.467484e+12,1.351694e+12
Western Europe,8.8,27368.8,26653.6,365802.4,73973.6,13.6140,176.2,8.646200e+09,134.00,23.431620,2.246361e+12,2.572350e+12,2.719032e+12,2.435150e+12,2.413790e+12,2.607178e+12,2.465591e+12,2.565183e+12,2.665684e+12,2.350811e+12


In [116]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    top15 = merged()
    ax = top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#e41a1c','#4daf4a','#4daf4a','#4daf4a','#377eb8',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#4daf4a','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(top15.index):
        ax.annotate(txt, [top15['Rank'][i], top15['% Renewable'][i]], ha='center')

    print("A bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

plot_optional()

KeyError: 'Country'